# Problem 1

We saw how to solve TSPs in this module and in particular presented two approaches to encode a TSP as an integer linear program. In this problem, we will ask you to adapt the TSP solution to the related problem of $k$ Travelling Salespeople Problem ($k$-TSP).

Let $G$ be a complete graph with $n$ vertices that we will label $0, \ldots, n-1$ (keeping Python array indexing in mind). Our costs are specified using a matrix $C$ wherein $C_{i,j}$ is the cost of the edge from vertex $i$ to $j$ for $i \not= j$.

In this problem, we have $k \geq 1$ salespeople who must  start from vertex $0$ of the graph (presumably the location of the sales office) and together visit every location in the graph,  each returning back to vertex $0$. Each location must be visited exactly once by some salesperson in the team. Therefore, other than vertex $0$ (the start/end vertex of every salesperson's tour), no two salesperson tours have a vertex in common. Notice that for $k=1$, this is just the regular TSP problem we have studied. 

Also, all $k$ salespeople must be employed in the tour. In other words, if we have $k=3$ then each salesperson must start at $0$ and visit a sequence of one or more vertices and come back to $0$. No salesperson can be "idle".

## Example-1

Consider a graph with $5$ nodes and the following cost matrix:

$$ \begin{array}{c|ccccc}
  \text{Vertices} & 0 & 1 & 2 & 3 & 4 \\ 
   \hline
 0 & - & 3 & 4 & 3 & 5 \\ 
 1 & 1 & - & 2 & 4 & 1 \\ 
 2 & 2 & 1 & - & 5 & 4 \\ 
 3 & 1 & 1 & 5 & - & 4 \\ 
 4 & 2 & 1 & 3 & 5 & - \\ 
 \end{array}$$
 
 For instance $C_{2,3}$ the cost of edge from vertex $2$ to $3$ is $5$. The $-$ in the diagonal entries simply tells us that we do not care what goes in there since we do not have self-loops in the graph.
 
The optimal $2$-TSP tour for $k=2$ salespeople is shown below.
  - Salesperson # 1: $0 \rightarrow 2 \rightarrow 1 \rightarrow 4 \rightarrow 0$.
  - Salesperson # 2: $0 \rightarrow 3 \rightarrow 0$.
  
The total cost of the edges traversed by the two salespeople equals $12$.

For $k=3$, the optimal $3-$ TSP tour is as shown below.
  - Salesperson # 1: $0 \rightarrow 1 \rightarrow 4$, 
  - Salesperson # 2: $0 \rightarrow 2$, 
  - Salesperson # 3: $0 \rightarrow 3$.

The total cost is $16$.

The objective of this problem is to formulate an ILP using the MTZ approach.

### Problem 1A (MTZ approach)

We will use the same ILP setup as in our notes (see the notes on Exact Approaches to TSP that includes the ILP encodings we will use in this problem).
  - Decision variables $x_{i,j}$ for $i \not= j$ denoting that the tour traverses the edge from $i$ to $j$.
  - Time stamps $t_1, \ldots, t_{n-1}$. The start/end vertex $0$ does not get a time stamp.
  
Modify the MTZ approach to incorporate the fact that $k$ salespeople are going to traverse the graph.

#### (A) Degree Constraints

What do the new degree constraints look like? Think about how many edges in the tour will need to enter/leave each vertex? Note that you may have to treat vertex $0$ differently from the other vertices of the graph.

Your answer below is not graded. However you are encouraged to write it down and check with the answers to select problems provided at the end.

YOUR ANSWER HERE

### (B) Time Stamp Constraints 

Formulate the time stamp constraints for the $k$-TSP problem. Think about how you would need to change them to eliminate subtour.

Your answer below is not graded. However you are encouraged to write it down and check with the answers to select problems provided at the end.


YOUR ANSWER HERE

### (C) Implement

Complete the implementation of the function `k_tsp_mtz_encoding(n, k, cost_matrix)` below. It follows the same input convention as the code supplied in the notes. The input `n` denotes the size of the graph with vertices labeled `0`,.., `n-1`, `k` is the number of salespeople, and `cost_matrix` is a list of lists wherein `cost_matrix[i][j]` is the edge cost to go from `i` to `j` for `i != j`. Your code must avoid accessing `cost_matrix[i][i]` to avoid bugs. These entries will be supplied as `None` in the test cases.

Your code must return a list `lst` that has exactly $k$ lists in it, wherein `lst[j]` represents the locations visited by the $j^{th}$ salesperson. 

For the example above, for $k=2$, your code must return 
~~~
[ [0, 2, 1, 4], [0, 3] ]
~~~
For the example above, for $k=3$, your code must return
~~~
[ [0, 1, 4], [0, 2], [0, 3] ]
~~~

In [54]:
from pulp import *

def recurList(sol,lst,x,n): 
    for i in range(n):
        if sol[x*n+i]==1:
            if i==0:
                return lst
            else:
                lst.append(i)
                return recurList(sol,lst,i,n)
            

def k_tsp_mtz_encoding(n, k, cost_matrix):
    # check inputs are OK
    assert 1 <= k < n
    assert len(cost_matrix) == n, f'Cost matrix is not {n}x{n}'
    assert all(len(cj) == n for cj in cost_matrix), f'Cost matrix is not {n}x{n}'
    prob = LpProblem('kTSP', LpMinimize)
    # finish your implementation here
    # your code must return a list of k-lists [ [0, i1, ..., il], [0, j1,...,jl],...] wherein 
    # the ith entry in our list of lists represents the 
    # tour undertaken by the ith salesperson
    # your code here
    #raise NotImplementedError
    for i in range(n):
        cost_matrix[i][i]=0
    vars=[pulp.LpVariable(f'x{i}{j}', lowBound=0,upBound=1, cat='Integer') for i in range(n) for j in range(n)]
    if k==k: #kill subtours
        ts=[pulp.LpVariable(f't{i}', lowBound=0,upBound=n, cat='Integer') for i in range(n)]
        
    prob += lpSum([cost_matrix[i][j]*vars[i*n+j] for i in range(n) for j in range(n)])
    for i in range(n):
        prob+=vars[i*(n+1)]==0 # diagonal=0
        if i ==0:
            # out
            prob += lpSum([vars[j+i*n] for j in range(n)])==k
            # in
            prob += lpSum([vars[j*n+i] for j in range(n)])==k
        else:
            # out
            prob += lpSum([vars[j+i*n] for j in range(n)])==1
            # in
            prob += lpSum([vars[j*n+i] for j in range(n)])==1
    if k==k:
        for i in range(n):
            for j in range(n):
                if i!=j and (i!=0 and j!=0):
                    prob +=ts[j]>=ts[i]+1 - (2*n)*(1-vars[i*n+j]) 
                
    prob.solve()
    sol=[vars[i*n+j].varValue for i in range(n) for j in range(n)]
    status=prob.status
    #print(sol)
    for i in range(n):
        print(sol[0+i*n:i*n+n])
    #convert to the matrix of matrix
    res=[]
    for i in range(n):
        if sol[i]==1:
            res.append([0,i])
            
    print("first dests:",res) # ok
    for i in range(len(res)):
        lst=res[i]
        res[i]=recurList(sol,lst,lst[-1],n)
    return res
        
  

In [55]:
cost_matrix=[ [None,3,4,3,5],
             [1, None, 2,4, 1],
             [2, 1, None, 5, 4],
             [1, 1, 5, None, 4],
             [2, 1, 3, 5, None] ]
n=5
k=2
all_tours = k_tsp_mtz_encoding(n, k, cost_matrix)
print(f'Your code returned tours: {all_tours}')
assert len(all_tours) == k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]

print(f'Tour cost obtained by your code: {tour_cost}')
assert abs(tour_cost - 12) <= 0.001, f'Expected tour cost is 12, your code returned {tour_cost}'
for i in range(1, n):
    is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
    assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'

print('test passed: 3 points')


[0.0, 0.0, 1.0, 1.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 1.0]
[0.0, 1.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0]
first dests: [[0, 2], [0, 3]]
Your code returned tours: [[0, 2, 1, 4], [0, 3]]
Tour cost obtained by your code: 12
test passed: 3 points


In [56]:
cost_matrix=[ [None,3,4,3,5],
             [1, None, 2,4, 1],
             [2, 1, None, 5, 4],
             [1, 1, 5, None, 4],
             [2, 1, 3, 5, None] ]
n=5
k=3
all_tours = k_tsp_mtz_encoding(n, k, cost_matrix)
print(f'Your code returned tours: {all_tours}')
assert len(all_tours) == k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]

print(f'Tour cost obtained by your code: {tour_cost}')
assert abs(tour_cost - 16) <= 0.001, f'Expected tour cost is 16, your code returned {tour_cost}'
for i in range(1, n):
    is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
    assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'

print('test passed: 2 points')

[0.0, 1.0, 1.0, 1.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 1.0]
[1.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0]
first dests: [[0, 1], [0, 2], [0, 3]]
Your code returned tours: [[0, 1, 4], [0, 2], [0, 3]]
Tour cost obtained by your code: 16
test passed: 2 points


In [57]:
cost_matrix = [ 
 [None, 1, 1, 1, 1, 1, 1, 1],
    [0, None, 1, 2, 1, 1, 1, 1],
    [1, 0, None, 1, 2, 2, 2, 1],
    [1, 2, 2, None, 0, 1, 2, 1],
    [1, 1, 1, 1, None, 1, 1, 1],
    [0,  1, 2, 1, 1, None, 1, 1],
    [1, 0,  1, 2, 2, 2,None, 1],
    [1, 2, 2, 0, 1, 2, 1, None],
]
n = 8
k = 2

all_tours = k_tsp_mtz_encoding(n, k, cost_matrix)
print(f'Your code returned tours: {all_tours}')
assert len(all_tours) == k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]

print(f'Tour cost obtained by your code: {tour_cost}')
assert abs(tour_cost - 4) <= 0.001, f'Expected tour cost is 4, your code returned {tour_cost}'
for i in range(1, n):
    is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
    assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'

print('test passed: 3 points')

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
first dests: [[0, 6], [0, 7]]
Your code returned tours: [[0, 6, 2, 1], [0, 7, 3, 4, 5]]
Tour cost obtained by your code: 4
test passed: 3 points


In [58]:
cost_matrix = [ 
 [None, 1, 1, 1, 1, 1, 1, 1],
    [0, None, 1, 2, 1, 1, 1, 1],
    [1, 0, None, 1, 2, 2, 2, 1],
    [1, 2, 2, None, 0, 1, 2, 1],
    [1, 1, 1, 1, None, 1, 1, 1],
    [0,  1, 2, 1, 1, None, 1, 1],
    [1, 0,  1, 2, 2, 2,None, 1],
    [1, 2, 2, 0, 1, 2, 1, None],
]
n = 8
k = 4

all_tours = k_tsp_mtz_encoding(n, k, cost_matrix)
print(f'Your code returned tours: {all_tours}')
assert len(all_tours) == k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]

print(f'Tour cost obtained by your code: {tour_cost}')
assert abs(tour_cost - 6) <= 0.001, f'Expected tour cost is 6, your code returned {tour_cost}'
for i in range(1, n):
    is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
    assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'

print('test passed: 2 points')

[0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
first dests: [[0, 2], [0, 5], [0, 6], [0, 7]]
Your code returned tours: [[0, 2], [0, 5], [0, 6, 1], [0, 7, 3, 4]]
Tour cost obtained by your code: 6
test passed: 2 points


In [59]:
from random import uniform, randint

def create_cost(n):
    return [ [uniform(0, 5) if i != j else None for j in range(n)] for i in range(n)]

for trial in range(5):
    print(f'Trial # {trial}')
    n = randint(5, 11)
    k = randint(2, n//2)
    print(f' n= {n}, k={k}')
    cost_matrix = create_cost(n)
    print('cost_matrix = ')
    print(cost_matrix)
    all_tours = k_tsp_mtz_encoding(n, k, cost_matrix)
    print(f'Your code returned tours: {all_tours}')
    assert len(all_tours) == k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

    tour_cost = 0
    for tour in all_tours:
        assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
        i = 0
        for j in tour[1:]:
            tour_cost += cost_matrix[i][j]
            i = j
        tour_cost += cost_matrix[i][0]

    print(f'Tour cost obtained by your code: {tour_cost}')
    #assert abs(tour_cost - 6) <= 0.001, f'Expected tour cost is 6, your code returned {tour_cost}'
    for i in range(1, n):
        is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
        assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'
    print('------')
print('test passed: 15 points')

Trial # 0
 n= 9, k=2
cost_matrix = 
[[None, 0.3771453705502009, 2.1975206342276525, 3.170477303375241, 2.162453644871467, 2.192675967777413, 4.394531973467768, 4.955808122329305, 4.385315836462644], [2.6687367430911006, None, 2.839969846304622, 2.670754146765042, 2.1458150582647995, 3.831507348432588, 2.3757454138852347, 4.752142169355633, 0.8949383733528643], [0.9476243132356088, 3.6370745937043236, None, 1.8356570535813443, 1.6460861954010326, 1.1616839657139004, 4.955684401958978, 2.9859909921570367, 1.9433610122719547], [4.290038844272789, 0.5346164498556782, 0.41574966000874736, None, 3.339675811224212, 0.4354985294705016, 0.831735712689583, 2.7676978013600264, 4.636576838416438], [3.583543690023119, 2.3789276961234105, 4.397615958261236, 1.467465914083042, None, 4.9743864564342735, 2.1645904052132776, 0.34406743109830384, 1.9992256074089143], [0.9160216185713338, 0.446396928878367, 2.3130323845937015, 3.5256715555138336, 0.8272321023321844, None, 4.890021667647822, 2.764597870511

## Problem 1 B

Notice that in previous part, it happens that with $k=4$ salespeople, we actually get a worse cost than using $k=3$ people. You can try out a few examples to convince yourself as to why this happens. 

We wish to modify the problem to allow salespeople to idle. In other words, although we input $k$ salespeople, the tour we construct may involve $1 \leq l \leq k$ salespeople. 

Modify the ILP formulation from the previous problem to solve the problem of up to $k$ people rather than exactly $k$ salespeople. Note that we still require that every vertex be visited exactly once by some salesperson. 

Complete the implementation of the function `upto_k_tsp_mtz_encoding(n, k, cost_matrix)` below. It follows the same input convention as previous problem but note that we are now computing a tour with at most $k$ salespeople. In other words, not all salespeople need be employed in the tour.

Your code must return a list `lst` that has less than or equal to  $k$ lists, wherein `lst[j]` represents the locations visited by the $j^{th}$ salesperson. 

For Example-1 from the previous part above, for $k=2$ or $k=3$, your code must return 
~~~
[ [0, 3, 1, 4, 2] ]
~~~
As it turns out, in this example a single salesperson suffices to yield optimal cost.

In [60]:
from pulp import *
def calculateCost(k_mat,cost_matrix):
    tot=0
    for mat in k_mat:
        tot0=0
        for idx in range(len(mat)):
            if idx+1==len(mat):
                tot0+=cost_matrix[mat[idx]][0] # going back to 0
            else:
                tot0+=cost_matrix[mat[idx]][mat[idx+1]] # all paths from 0 till the scd last node
        tot+=tot0
    return tot

def upto_k_tsp_mtz_encoding(n, k, cost_matrix):
    # check inputs are OK
    assert 1 <= k < n
    assert len(cost_matrix) == n, f'Cost matrix is not {n}x{n}'
    assert all(len(cj) == n for cj in cost_matrix), f'Cost matrix is not {n}x{n}'
    prob = LpProblem('kTSP', LpMinimize)
    # finish your implementation here
    # your code must return a list of k-lists [ [0, i1, ..., il], [0, j1,...,jl],...] wherein 
    # the ith entry in our list of lists represents the 
    # tour undertaken by the ith salesperson
    # your code here
    #raise NotImplementedError
    for i in range(1,k,1):
        print("test for ",i)
        if i==1:
            res_f=k_tsp_mtz_encoding(n, i, cost_matrix)
            res=res_f
            cost_f=calculateCost(res_f,cost_matrix)
            cost=cost_f
            k_f=1
        else:
            res=k_tsp_mtz_encoding(n, i, cost_matrix)
            cost=calculateCost(res,cost_matrix)
            if cost<=cost_f:
                cost_f=cost
                res_f=res
                k_f=i
    return res_f
    
        
cost_matrix=[ [None,3,4,3,5],
             [1, None, 2,4, 1],
             [2, 1, None, 5, 4],
             [1, 1, 5, None, 4],
             [2, 1, 3, 5, None] ]
n=5
k=3
all_tours = upto_k_tsp_mtz_encoding(n, k, cost_matrix)
print(f'Your code returned tours: {all_tours}')
assert len(all_tours) <= k, f'<= {k} tours -- your code returns {len(all_tours)} tours instead'

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]

assert len(all_tours) == 1, f'In this example, just one salesperson is needed to optimally visit all vertices. Your code returns {len(all_tours)}'
print(f'Tour cost obtained by your code: {tour_cost}')        

test for  1
[0.0, 0.0, 0.0, 1.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 1.0]
[1.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 0.0, 0.0]
first dests: [[0, 3]]
test for  2
[0.0, 0.0, 1.0, 1.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 1.0]
[0.0, 1.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0]
first dests: [[0, 2], [0, 3]]
Your code returned tours: [[0, 3, 1, 4, 2]]
Tour cost obtained by your code: 10


In [61]:
cost_matrix=[ [None,3,4,3,5],
             [1, None, 2,4, 1],
             [2, 1, None, 5, 4],
             [1, 1, 5, None, 4],
             [2, 1, 3, 5, None] ]
n=5
k=3
all_tours = upto_k_tsp_mtz_encoding(n, k, cost_matrix)
print(f'Your code returned tours: {all_tours}')
assert len(all_tours) <= k, f'<= {k} tours -- your code returns {len(all_tours)} tours instead'

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]

assert len(all_tours) == 1, f'In this example, just one salesperson is needed to optimally visit all vertices. Your code returns {len(all_tours)}'
print(f'Tour cost obtained by your code: {tour_cost}')
assert abs(tour_cost - 10) <= 0.001, f'Expected tour cost is 10, your code returned {tour_cost}'
for i in range(1, n):
    is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
    assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'

print('test passed: 3 points')

test for  1
[0.0, 0.0, 0.0, 1.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 1.0]
[1.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 0.0, 0.0]
first dests: [[0, 3]]
test for  2
[0.0, 0.0, 1.0, 1.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 1.0]
[0.0, 1.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0]
first dests: [[0, 2], [0, 3]]
Your code returned tours: [[0, 3, 1, 4, 2]]
Tour cost obtained by your code: 10
test passed: 3 points


In [62]:
cost_matrix = [ 
 [None, 1, 1, 1, 1, 1, 1, 1],
    [0, None, 1, 2, 1, 1, 1, 1],
    [1, 0, None, 1, 2, 2, 2, 1],
    [1, 2, 2, None, 0, 1, 2, 1],
    [1, 1, 1, 1, None, 1, 1, 1],
    [0,  1, 2, 1, 1, None, 1, 1],
    [1, 0,  1, 2, 2, 2,None, 1],
    [1, 2, 2, 0, 1, 2, 1, None],
]
n = 8
k = 5

all_tours = upto_k_tsp_mtz_encoding(n, k, cost_matrix)
print(f'Your code returned tours: {all_tours}')
assert len(all_tours) <= k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]

print(f'Tour cost obtained by your code: {tour_cost}')
assert abs(tour_cost - 4) <= 0.001, f'Expected tour cost is 4, your code returned {tour_cost}'
for i in range(1, n):
    is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
    assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'

print('test passed: 3 points')

test for  1
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
first dests: [[0, 6]]
test for  2
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
first dests: [[0, 6], [0, 7]]
test for  3
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 1.0,

In [63]:
from random import uniform, randint

def create_cost(n):
    return [ [uniform(0, 5) if i != j else None for j in range(n)] for i in range(n)]

for trial in range(20):
    print(f'Trial # {trial}')
    n = randint(5, 11)
    k = randint(2, n//2)
    print(f' n= {n}, k={k}')
    cost_matrix = create_cost(n)
    print('cost_matrix = ')
    print(cost_matrix)
    all_tours = upto_k_tsp_mtz_encoding(n, k, cost_matrix)
    print(f'Your code returned tours: {all_tours}')
    assert len(all_tours) <= k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

    tour_cost = 0
    for tour in all_tours:
        assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
        i = 0
        for j in tour[1:]:
            tour_cost += cost_matrix[i][j]
            i = j
        tour_cost += cost_matrix[i][0]

    print(f'Tour cost obtained by your code: {tour_cost}')
    #assert abs(tour_cost - 6) <= 0.001, f'Expected tour cost is 6, your code returned {tour_cost}'
    for i in range(1, n):
        is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
        assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'
    print('------')
print('test passed: 4 points')

Trial # 0
 n= 11, k=5
cost_matrix = 
[[None, 1.488063842767004, 4.609033690230404, 3.0977705897477756, 1.742777637163135, 3.622753653384013, 3.0294500389349532, 4.533275728116961, 2.13543093687273, 3.2385682983549176, 4.058489959931891], [3.727222832990642, None, 0.21617177974634139, 3.9304771043237174, 0.21001990866282572, 0.3549196309414415, 2.5797775557289695, 4.888935661585509, 1.3807326014420174, 2.6462504272403518, 2.2330272172360974], [4.3831371311737515, 2.190045493698349, None, 0.9385561353285482, 3.8521226907061696, 4.427765621652882, 2.87046218981096, 1.6620374179247077, 3.219136199700803, 1.3880494961408711, 3.769747967796104], [0.020419748347274003, 3.9376626210893284, 2.150224007137597, None, 1.1145832779344234, 3.5728758804768503, 3.9482520306213758, 0.5829333003737336, 1.9857322076086559, 1.1763597280007265, 4.328848356737527], [3.817185437925475, 0.41512020204221955, 4.32596593715374, 2.109749257486264, None, 0.4444478730249679, 1.023455319105725, 4.660422697971878, 1.

## Problem 2 (10 points)

We noted the use of Christofides algorithm for metric TSP. We noted that for non-metric TSPs it does not work. 
In fact, the shortcutting used in Christofides algorithm can be _arbitrarily_ bad for a TSP that is symmetric but fails to be a metric TSP.

In this example, we would like you to frame a symmetric TSP instance ($C_{ij} = C_{ji}$) with $5$ vertices wherein the algorithm obtained by "shortcutting" the minimum spanning tree (MST), that would be a 2-factor approximation for metric TSP, yields an answer that can be quite "far off" from the optimal solution.

Enter a __symmetric__ cost-matrix for the TSP below as a 5x5 matrix as a list of lists following convention in our notes. such that the optimal answer is at least $10^6$ times smaller than that obtained by the TSP-based approximation. We will test your answer by running the TSP with shortcutting algorithm.

__Hint:__ Force the edges $(0,1), (1,2), (2,3)$ and $(3,4)$ to be the minimum spanning tree. But make the weight of the edge form $4$ back to $0$ very high.


__Note:__ this problem is tricky and requires you to be very familiar with how Christofides algorithm works. It may be wise to attempt the remaining problems first before this one. Do not worry about the diagonal entry of your matrices.


In [65]:
# Write down the cost matrix as a list of lists. Remember, we want an example with $5$ vertices
cost_matrix = [[0,1,1,1,10**100], [1,0,1,1,1],[1,1,0,1,1],[1,1,1,0,1],[10**100,1,1,1,0]]
# The cost matrix should represent a TSP instance such that the optimal answer is 10^6 smaller than the 
# answer obtained by shortcutting a minimum spanning tree.
# your code here
#raise NotImplementedError


In [66]:
# check that the cost matrix is symmetric.
assert len(cost_matrix) == 5, f'Cost matrix must have 5 rows. Yours has {len(cost_matrix)} rows'
assert all(len(cj) == 5 for cj in cost_matrix), f'Each row of the cost matrix must have 5 entries.'
for i in range(5):
    for j in range(i):
        assert cost_matrix[i][j] == cost_matrix[j][i], f'Cost matrix fails to be symmetric at entries {(i,j)} and {(j,i)}'
print('Structure of your cost matrix looks OK (3 points).')

Structure of your cost matrix looks OK (3 points).


Please ensure that you run the two cells below or else, your tests will fail.

In [67]:
# MST based tsp approximation
import networkx as nx

# This code implements the simple MST based shortcutting approach that would yield factor of 2
# approximation for metric TSPs.
def minimum_spanning_tree_tsp(n, cost_matrix):
    G = nx.Graph()
    for i in range(n):
        for j in range(i):
            G.add_edge(i, j, weight=cost_matrix[i][j])
    T = nx.minimum_spanning_tree(G)
    print(f'MST for your graph has the edges {T.edges}')
    mst_cost = 0
    mst_dict = {} # store mst as a dictionary
    for (i,j) in T.edges:
        mst_cost += cost_matrix[i][j]
        if i in mst_dict:
            mst_dict[i].append(j)
        else:
            mst_dict[i] = [j]
        if j in mst_dict:
            mst_dict[j].append(i)
        else:
            mst_dict[j] = [i]
    print(f'MST cost: {mst_cost}')
    print(mst_dict)
    # Let's form a tour with short cutting
    def traverse_mst(tour_so_far, cur_node):
        assert cur_node in mst_dict
        next_nodes = mst_dict[cur_node]
        for j in next_nodes:
            if j in tour_so_far:
                continue
            tour_so_far.append(j)
            traverse_mst(tour_so_far, j)
        return
    tour = [0]
    traverse_mst(tour, 0)
    i = 0
    tour_cost = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]
    return tour, tour_cost

In [68]:
# optimal TSP tour taken from our notes using MTZ encoding
from pulp import *

def mtz_encoding_tsp(n, cost_matrix):
    assert len(cost_matrix) == n, f'Cost matrix is not {n}x{n}'
    assert all(len(cj) == n for cj in cost_matrix), f'Cost matrix is not {n}x{n}'
    # create our encoding variables
    binary_vars = [ # add a binary variable x_{ij} if i not = j else simply add None
        [ LpVariable(f'x_{i}_{j}', cat='Binary') if i != j else None for j in range(n)] 
        for i in range(n) ]
    # add time stamps for ranges 1 .. n (skip vertex 0 for timestamps)
    time_stamps = [LpVariable(f't_{j}', lowBound=0, upBound=n, cat='Continuous') for j in range(1, n)]
    # create the problem
    prob = LpProblem('TSP-MTZ', LpMinimize)
    # create add the objective function 
    objective_function = lpSum( [ lpSum([xij*cj if xij != None else 0 for (xij, cj) in zip(brow, crow) ])
                           for (brow, crow) in zip(binary_vars, cost_matrix)] )
    
    prob += objective_function 
    
    # add the degree constraints
    for i in range(n):
        # Exactly one leaving variable
        prob += lpSum([xj for xj in binary_vars[i] if xj != None]) == 1
        # Exactly one entering
        prob += lpSum([binary_vars[j][i] for j in range(n) if j != i]) == 1
    # add time stamp constraints
    for i in range(1,n):
        for j in range(1, n):
            if i == j: 
                continue
            xij = binary_vars[i][j]
            ti = time_stamps[i-1]
            tj = time_stamps[j -1]
            prob += tj >= ti + xij - (1-xij)*(n+1) # add the constraint
    # Done: solve the problem
    status = prob.solve(PULP_CBC_CMD(msg=False)) # turn off messages
    assert status == constants.LpStatusOptimal, f'Unexpected non-optimal status {status}'
    # Extract the tour
    tour = [0]
    tour_cost = 0
    while len(tour) < n:
        i = tour[-1]
        # find all indices j such that x_ij >= 0.999 
        sols = [j for (j, xij) in enumerate(binary_vars[i]) if xij != None and xij.varValue >= 0.999]
        assert len(sols) == 1, f'{sols}' # there better be just one such vertex or something has gone quite wrong
        j = sols[0] # extract the lone solutio 
        tour_cost = tour_cost + cost_matrix[i][j] # add to the tour cost
        tour.append(j) # append to the tour
        assert j != 0
    i = tour[-1]
    tour_cost = tour_cost + cost_matrix[i][0]
    return tour, tour_cost
        

In [69]:
#test that exact answer is 10^6 times smaller than approximate answer.
# compute MST based approximation
tour, tour_cost = minimum_spanning_tree_tsp(5, cost_matrix)
print(f'MST approximation yields tour is {tour} with cost {tour_cost}')
# compute exact answer
opt_tour, opt_tour_cost = mtz_encoding_tsp(5, cost_matrix)
print(f'Optimal tour is {opt_tour} with cost {opt_tour_cost}')
# check that the fraction is 1million times apart.
assert tour_cost/opt_tour_cost >= 1E+06, 'The TSP + shortcutting tour must be at least 10^6 times costlier than optimum. In your case, the ratio is {tour_cost/opt_tour_cost}'
print('Test passed: 7 points')

MST for your graph has the edges [(1, 0), (1, 2), (1, 3), (1, 4)]
MST cost: 4
{1: [0, 2, 3, 4], 0: [1], 2: [1], 3: [1], 4: [1]}
MST approximation yields tour is [0, 1, 2, 3, 4] with cost 10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000004
Optimal tour is [0, 1, 4, 3, 2] with cost 5
Test passed: 7 points


## Problem 3

In this problem, we wish to solve TSP with additional constraints. Suppose we are given a TSP instance in the form of a $n\times n$ matrix $C$ representing a complete graph. 

We wish to solve a TSP but with additional constraints specified as a list $[(i_0, j_0), \ldots, (i_k, j_k)]$ wherein each pair $(i_l, j_l)$ in the list specifies that vertex $i_l$ must be visited in the tour before vertex $j_l$. Assume that the tour starts/ends at vertex $0$ and none of the vertices in the constraint list is $0$. I.e, $i_l\not= 0, j_l \not= 0$ for all $0 \leq l \leq k$.

Modify one of the ILP encodings we have presented to solve TSP with extra constraints. Implement your solution in the function `tsp_with_extra_constraints(n, cost_matrix, constr_list)` where the extra argument `constr_list` is a list of pairs `[(i0,j0),...., (ik, jk)]` that specify for each pair `(il,jl)` that vertex `il` must be visited before `jl`. Assume that the problem is feasible (no need to handle infeasible instances). 
Your code should output the optimal tour as a list.

## Example

Consider again the graph with $5$ nodes and the following cost matrix from problem 1:

$$ \begin{array}{c|ccccc}
  \text{Vertices} & 0 & 1 & 2 & 3 & 4 \\ 
   \hline
 0 & - & 3 & 4 & 3 & 5 \\ 
 1 & 1 & - & 2 & 4 & 1 \\ 
 2 & 2 & 1 & - & 5 & 4 \\ 
 3 & 1 & 1 & 5 & - & 4 \\ 
 4 & 2 & 1 & 3 & 5 & - \\ 
 \end{array}$$
 
The optimal TSP tour will be $[0, 3, 1, 4, 2]$ with total cost $10$.

Suppose we added the constraints $[(4, 3), (1, 2)]$ we note that the tour satisfies the constraint $(1, 2)$ since it visits vertex $1$ before vertex $2$ but it unfortunately, $(4,3)$ is violated since vetex $3$ is visited before $4$ in the tour.


In [72]:
def tsp_with_extra_constraints(n, cost_matrix, constraints):
    assert len(cost_matrix) == n, f'Cost matrix is not {n}x{n}'
    assert all(len(cj) == n for cj in cost_matrix), f'Cost matrix is not {n}x{n}'
    assert all( 1 <= i < n and 1 <= j < n and i != j for (i,j) in constraints)
    # TODO: encode the problem in pulp (a) decision variables; (b) constraints; (c) objective; (d) solve and extract
    #       solution. This is going to be very close to the MTZ encoding that we have presented in our notes. You can use
    #       our code as a starting point.
    # your code here
    #raise NotImplementedError
    prob = LpProblem('kTSP', LpMinimize)
    # finish your implementation here
    # your code must return a list of k-lists [ [0, i1, ..., il], [0, j1,...,jl],...] wherein 
    # the ith entry in our list of lists represents the 
    # tour undertaken by the ith salesperson
    # your code here
    # raise NotImplementedError
    # I will just copy my answer from 1A and modify the whole sht, bruh
    k=1
    for i in range(n):
        cost_matrix[i][i]=0
    vars=[pulp.LpVariable(f'x{i}{j}', lowBound=0,upBound=1, cat='Integer') for i in range(n) for j in range(n)]
    if k==k: #kill subtours
        ts=[pulp.LpVariable(f't{i}', lowBound=0,upBound=n, cat='Integer') for i in range(n)]
        
    prob += lpSum([cost_matrix[i][j]*vars[i*n+j] for i in range(n) for j in range(n)])
    for i in range(n):
        prob+=vars[i*(n+1)]==0 # diagonal=0
        if i ==0:
            # out
            prob += lpSum([vars[j+i*n] for j in range(n)])==k
            # in
            prob += lpSum([vars[j*n+i] for j in range(n)])==k
        else:
            # out
            prob += lpSum([vars[j+i*n] for j in range(n)])==1
            # in
            prob += lpSum([vars[j*n+i] for j in range(n)])==1
    if k==k:
        for i in range(n):
            for j in range(n):
                if i!=j and (i!=0 and j!=0):
                    prob +=ts[j]>=ts[i]+1 - (2*n)*(1-vars[i*n+j]) 
    
    #additional conditions
    for con in constraints:
        prob +=ts[con[0]]<=ts[con[1]]
    
    prob.solve()
    sol=[vars[i*n+j].varValue for i in range(n) for j in range(n)]
    status=prob.status
    #print(sol)
    for i in range(n):
        print(sol[0+i*n:i*n+n])
    #convert to the matrix of matrix
    res=[]
    for i in range(n):
        if sol[i]==1:
            res.append([0,i])
            
    print("first dests:",res) # ok
    for i in range(len(res)):
        lst=res[i]
        res[i]=recurList(sol,lst,lst[-1],n)
    return res[0]   

In [73]:
cost_matrix=[ [None,3,4,3,5],
             [1, None, 2,4, 1],
             [2, 1, None, 5, 4],
             [1, 1, 5, None, 4],
             [2, 1, 3, 5, None] ]
n=5
constraints = [(3,4),(1,2)]
tour = tsp_with_extra_constraints(n, cost_matrix, constraints)
i = 0
tour_cost = 0
for j in tour[1:]:
    tour_cost += cost_matrix[i][j]
    i = j
tour_cost += cost_matrix[i][0]
print(f'Tour:{tour}')
print(f'Cost of your tour: {tour_cost}')
assert abs(tour_cost-10) <= 0.001, 'Expected cost was 10'
for i in range(n):
    num = sum([1 if j == i else 0 for j in tour])
    assert  num == 1, f'Vertex {i} repeats {num} times in tour'
for (i, j) in constraints:
    assert tour.index(i) < tour.index(j), f'Tour does not respect constraint {(i,j)}'
print('Test Passed (3 points)')

[0.0, 0.0, 0.0, 1.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 1.0]
[1.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 0.0, 0.0]
first dests: [[0, 3]]
Tour:[0, 3, 1, 4, 2]
Cost of your tour: 10
Test Passed (3 points)


In [74]:
cost_matrix=[ [None,3,4,3,5],
             [1, None, 2,4, 1],
             [2, 1, None, 5, 4],
             [1, 1, 5, None, 4],
             [2, 1, 3, 5, None] ]
n=5
constraints = [(4,3),(1,2)]
tour = tsp_with_extra_constraints(n, cost_matrix, constraints)
i = 0
tour_cost = 0
for j in tour[1:]:
    tour_cost += cost_matrix[i][j]
    i = j
tour_cost += cost_matrix[i][0]
print(f'Tour:{tour}')
print(f'Cost of your tour: {tour_cost}')
assert abs(tour_cost-13) <= 0.001, 'Expected cost was 13'
for i in range(n):
    num = sum([1 if j == i else 0 for j in tour])
    assert  num == 1, f'Vertex {i} repeats {num} times in tour'
for (i, j) in constraints:
    assert tour.index(i) < tour.index(j), f'Tour does not respect constraint {(i,j)}'
print('Test Passed (3 points)')

[0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 1.0]
[0.0, 0.0, 0.0, 1.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 0.0, 0.0]
first dests: [[0, 1]]
Tour:[0, 1, 4, 2, 3]
Cost of your tour: 13
Test Passed (3 points)


In [75]:
from random import uniform, randint

def create_cost(n):
    return [ [uniform(0, 5) if i != j else None for j in range(n)] for i in range(n)]

for trial in range(20):
    print(f'Trial # {trial}')
    n = randint(6, 11)
    cost_matrix = create_cost(n)
    constraints = [(1, 3), (4, 2), (n-1, 1), (n-2, 2)]
    tour = tsp_with_extra_constraints(n, cost_matrix, constraints)
    i = 0
    tour_cost = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]
    print(f'Tour:{tour}')
    print(f'Cost of your tour: {tour_cost}')
    for i in range(n):
        num = sum([1 if j == i else 0 for j in tour])
        assert  num == 1, f'Vertex {i} repeats {num} times in tour'
    for (i, j) in constraints:
        assert tour.index(i) < tour.index(j), f'Tour does not respect constraint {(i,j)}'
print('Test Passed (10 points)')

Trial # 0
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
first dests: [[0, 5]]
Tour:[0, 5, 6, 1, 3, 4, 2]
Cost of your tour: 12.997319399951083
Trial # 1
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
first dests: [[0, 6]]
Tour:[0, 6, 9, 1, 5, 8, 7, 4, 2, 3]
Cost of your tour: 12.484873035186578
Trial # 2
[0.0, 0.0, 0.0, 0.0, 0.0,

## Answers to Select Problems

### 1A part A

- Vertex 0: $k$ edges leave and $k$ edges enter.
- Vertex 1, ..., n-1: 1 edge leaves and 1 edge enters (same as TSP).

### 1A part B

This is a trick question. There is no need to change any of the time stamp related constraints.


## That's All Folks!